## prepare_data.py

In [38]:
from pathlib import Path
from config import get_config
# from data.data_pipe import load_bin, load_mx_rec
import argparse
from easydict import EasyDict
from torchvision import transforms as trans
import os
import pickle
import bcolz
import numpy as np
import mxnet as mx
import cv2
from pathlib import Path
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import cv2
import bcolz
import pickle
import torch
import mxnet as mx
from tqdm import tqdm





### data_pipe.py

In [39]:
def load_bin(path, rootdir, transform, image_size=[112,112]):
    if not rootdir.exists():
        rootdir.mkdir()
    bins, issame_list = pickle.load(open(path, 'rb'), encoding='bytes')
    data = bcolz.fill([len(bins), 3, image_size[0], image_size[1]], dtype=np.float32, rootdir=rootdir, mode='w')
    for i in range(len(bins)):
        _bin = bins[i]
        img = mx.image.imdecode(_bin).asnumpy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = Image.fromarray(img.astype(np.uint8))
        data[i, ...] = transform(img)
        i += 1
        if i % 1000 == 0:
            print('loading bin', i)
    print(data.shape)
    np.save(str(rootdir)+'_list', np.array(issame_list))
    return data, issame_list

In [45]:
dir(trans)

['CenterCrop',
 'ColorJitter',
 'Compose',
 'FiveCrop',
 'Grayscale',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'Pad',
 'RandomAffine',
 'RandomApply',
 'RandomChoice',
 'RandomCrop',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomOrder',
 'RandomPerspective',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'functional',
 'transforms']

### prepare_data.py

In [48]:
args = EasyDict()
args.data_path = Path('')
args.rec_path = '../data/lfw'
args.test_transform = trans.Compose([
#                     trans.Resize((112,112)),
                    trans.ToTensor(),
                    trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                ])

rec_path = args.data_path/args.rec_path
bin_files = 'small_lfw'

In [49]:
load_bin(rec_path/(bin_files+'.bin'),
         rec_path/bin_files, 
         args.test_transform)

(198, 3, 112, 112)


(carray((198, 3, 112, 112), float32)
   nbytes := 28.42 MB; cbytes := 22.10 MB; ratio: 1.29
   cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
   chunklen := 3; chunksize: 451584; blocksize: 262144
   rootdir := '..\data\lfw\small_lfw'
   mode    := 'w'
 [[[[-0.69411767 -0.69411767 -0.69411767 ..., -0.94509804 -0.94509804
     -0.94509804]
    [ 0.60000002  0.60784316  0.6156863  ..., -0.78039217 -0.79607844
     -0.78039217]
    [ 0.70980394  0.74901962  0.81176472 ..., -0.78039217 -0.78039217
     -0.7647059 ]
    ..., 
    [-0.7647059  -0.8509804  -0.88235295 ..., -0.95294118 -0.95294118
     -0.95294118]
    [-0.77254903 -0.84313726 -0.87450981 ..., -0.95294118 -0.95294118
     -0.95294118]
    [-0.78039217 -0.84313726 -0.87450981 ..., -0.95294118 -0.95294118
     -0.95294118]]
 
   [[-0.67843139 -0.68627453 -0.69411767 ..., -0.92941177 -0.92941177
     -0.92941177]
    [ 0.63921571  0.63137257  0.62352943 ..., -0.65490198 -0.66274512
     -0.64705884]
    [ 0.7803